In [ ]:
# !pip install -r https://raw.githubusercontent.com/EluciDATALab/elucidatalab.starterkits/main/notebooks/SK_3_3_Feature_Engineering/requirements.txt
!pip install -r https://raw.githubusercontent.com/EluciDATALab/elucidatalab.starterkits/SK_3_3_Feature_Engineering/notebooks/SK_3_3_Feature_Engineering/requirements.txt

In [ ]:
import starterkits.starterkit_3_3.support as sp
import starterkits.starterkit_3_3.visualization as vis

from pathlib import Path
DATA_PATH = '../../data/'

%matplotlib inline

# Starter Kit 3.3: Feature Engineering

## Description

Most data mining and machine learning algorithms do not work well if you just feed them your raw data: such data often contains noise and the most relevant distinguishing information is often implicit. For example, raw sensor data just contains a timestamp and a corresponding value, while interesting aspects of it might be the trends contained within or the number of times a threshold is exceeded.

Feature engineering is the process of extracting from the raw data the most relevant distinguishing characteristics that will be presented to the algorithm during modeling. It is a way of deriving new information from the existing data, so that its characteristics are more explicitly represented. The resulting features are eventually feed to the AI/ML algorithm of the model. In practice, the feature engineering step is achieved by the selection of the most appropriate parameters, or the composition of new features by manipulation, transformation and combination of the raw data.

Feature engineering is one of the most important and creative steps in the data science workflow. However, there is no clearly-defined formal process for engineering features and, consequently, this requires a lot of creativity, a good understanding of the domain and of the available data, some trial-and-error, etc. It is also desirable to have upfront an idea of the modeling and analysis task for which you want to use the resulting features, as this might help you to identify relevant features.


## Business goal

The overall goal of this Starter Kit is to present some **advanced feature engineering steps** related to **feature construction** and **extraction**. By keeping in mind **what is your business question** and **what is the corresponding data science task**, you will be able to derive **valuable features** that can be used in the next stage of your analysis.

## Application context

Feature engineering is one of the steps in the data science workflow with the most decisive impact on the accuracy of the model you want to develop. It is the final step before actually training the model and it defines how the input data will be fed to the model.

## Starter Kit outline

In this Starter Kit we use a dataset from the Regional Rail System of Pennsylvania. Before starting the feature extraction _per se_, we will first apply some basic preprocessing to the dataset and have a high-level overview of the data. We will then derive several interesting features from the dataset that can be used to characterise train rides and delays.

## Basic data understanding

In order to illustrate how to engineer features, we will use in this Starter Kit a dataset provided by the Southeastern Pennsylvania Transportation Authority (SEPTA), which can be downloaded [here](https://www.kaggle.com/septa/on-time-performance).

The SEPTA Regional Rail System consists of 13 branches and more than 150 stations in Philadelphia, Pennsylvania, and its suburbs and satellite cities, as depicted in the map below.

<img src="septaMap.png">

SEPTA uses On-Time Performance (OTP) to measure service reliability. OTP identifies the number of trains for all rail lines that arrive at their scheduled destination at the scheduled time. However, by industry standard, a train may arrive up to 5 minutes and 59 seconds after its scheduled time and still be considered on-time.

SEPTA has established an annual goal of 91% for Regional Rail On-Time Performance. Based on this goal, we will demonstrate how feature engineering can be used to validate SEPTA's requirement. More specifically, we will start designing features that may help us **predict** delays of a train on a given **stop** at a given **date**.

The SEPTA dataset consists of 2 CSV files that we will consider in this Starter Kit. The first file, `otp.csv`, provides OTP data, that is, information about trains and the times at which they arrive at the different stations on their lines. The table below shows an excerpt of this OTP dataset.

In [ ]:
df_otp = sp.load_otp_data(DATA_PATH=DATA_PATH)
df_otp.head(10)

The different attributes in this file OTP dataset are as follows:
* `train_id`: the identifier of the train
* `direction`: the train direction; its values are 'N' for Northbound and 'S' for Southbound
* `origin`: the station before *next_station*
* `next_station`: the next station stop at *timeStamp*
* `date`: the date of the journey
* `status`: the current train delay; its values are 'On Time' or the delay, i.e. the amount of time above the 5min59s limit until which a train is considered to be on time (e.g. '1 min', '5 min', '10 min'); a value of 999 indicates a suspended train
* `timeStamp`: the timestamp at which the train will arrive at `next_station`

The second file, `trainView.csv`, provides train tracking information:

In [ ]:
df_train_view = sp.load_train_view_data(DATA_PATH=DATA_PATH)
df_train_view.head(3)

Its most important attributes are:
* `lon`: the current GPS longitude of the train
* `lat`: the current GPS latitude of the train
* `timeStamp0`: the earliest timestamp at the current GPS coordinates
* `timeStamp1`: the latest timeStamp at the current GPS coordinates
* `seconds`: the time difference (in seconds) between `timeStamp1` and `timeStamp0`
* `track_change`: The name of the track if there was a track change, else -1

We can already observe in the previews of these two datasets format differences at the level of the `train_id` and `status` columns. We will address these in the next section.

## Data preprocessing


### Removing invalid data instances

A manual inspection of the dataset reveals that the `train_id` column from the OTP data contains 1 negative value and many values containing letters and punctuation characters. Let's filter out the rows for which the `train_id` is not a number.

In [ ]:
df_otp = sp.remove_invalid_data_instances(df=df_otp)

### Understanding train_id <a name="Understanding_train_id"></a>

`train_id` can be used an identifier for a specific train run between a departure and a destination station at a given time of the day. To confirm that this is possible, we count how many times a `train_id` passes through a given station on a given day in the `train_run` dataset:

In [ ]:
train_run = sp.get_train_run(df=df_otp)
print("In %0.2f%% of the cases, a train_id passes only once through a station on a given day." %
        (train_run.single_pass.sum() / float(len(train_run)) * 100))

We will exclude from the dataset the remaining 0.2% of cases, since they correspond to exceptional situations where a `train_id` was registered more than once in a day.
For consistency, we only keep in the trainView dataset those rows for which the (`train_id`, `next_station`, `date`) tuples also occur in the OTP dataset.

For the remainder of the notebook, we will refer to **train id** as an identifier for a train journey between a given _origin_ and a given _destination_ at a given _time_ of the day. In other words, a train id can be repeated on multiple days.
A **train run** on the other hand, is now defined as a train id on a specific date, as used in the trainview dataset.

In [ ]:
df_otp, df_train_view = sp.exclude_remaining_cases(otp=df_otp, train_view=df_train_view)

### Turning statuses into workable delays

The current string-based format of the `status` column in the OTP dataset is not suitable for performing calculations on the delays. We will transform its values into integers (in minutes) and rename the column to `delay`.

In [ ]:
df_otp = sp.calculate_delays_for_otp(df=df_otp)

We will also exclude missing (`None`) values present in the `status` column of the df_train_view dataset, and, as for the OTP dataset, convert the values of that column to integers and rename the column to `delay`.

In [ ]:
df_train_view = sp.calculate_delays_for_train_view(df=df_train_view)

### Removing suspended trains

Since a status of `999` represented a suspended train, we exclude those trains from the datasets.
In addition, a delay of 1440 minutes corresponds to one day delay, meaning that the train was most likely canceled. We exclude the trains with those delays from the datasets as well.

In [ ]:
df_otp, df_train_view = sp.remove_suspended_trains(df_otp=df_otp, df_train_view=df_train_view)

## Data overview

Let's check some basic characteristics of the (cleaned OTP) dataset.

In [ ]:
sp.get_otp_overview(df=df_otp)

As the table above indicates 74.9% of the trains being delayed, i.e. we demonstrate analytically that **the original requirement of having 91% of all trains on time** (definition: no more than 5min59s delay) **is far from being reached**. This is just a showcase of how feature engineering can be used in practice. In the remainder of the notebook we will present further examples to better understand the process.

## Feature engineering

We are now ready to investigate several features to demonstrate the workflow and some hidden obstacles while calculating features. You might come up with other features as well, so don't hesitate to try them out.

Based on our interest to characterize train rides and delays, we come up with the following features:
- [Day of the week](#Day_of_the_week)
- [Month of the year](#Month_of_the_year)
- [Rush hours](#Rush_hours)
- [Rank the different stops in a train run](#Rank_the_different_stops_in_a_train_run)
- [Distance between stations](#Distance_between_stations)
- [Cumulative distance along a train run](#Cumulative_distance_along_a__train__run_)
- [Total distance of a train run](#Total_distance_of_a__train__run_)
- [Northbound vs Southbound travels](#Northbound_vs_Southbound_travels)
- [Stations with big delay](#Stations_with_big_delay)
- [Delay over the last 7 days](#Delay_over_the_last_7_days)
- [Track changes](#Track_changes)

### Day of the week <a name="Day_of_the_week"></a>

Based on our collective experience from using public transport, we can expect that delays are more likely to happen at specific moments in a day (e.g. rush hours) and at specific days in a week (e.g. working days). In order to be able to easily verify these assumptions later on, we first extract for each entry in the OTP dataset the following features:
- the hour
- the name of the day
- the type of day, which we define as having 3 possible values: weekday, Saturday and Sunday
- and a boolean indicating whether that day is a weekday or a weekend day

Let's start by looking at the impact of day of the week on delay.

In [ ]:
vis.plot_train_delays(df=df_otp)

As we can see, the difference in delays on weekdays (Mon-Fri) is only minimal. We will thus categorize the day of the week into weekdays, saturdays and sundays.

In [ ]:
df_otp = sp.get_type_of_days(df=df_otp)

### Month of the year <a name="Month_of_the_year"></a>

Seasonal effects, expressed here as the month of the year, can have an impact on train performance. As an example consider the summer holidays one can expect less (commuting) traffic, albeit also reduced staff, which might affect how on-time trains are. Additionally, one might expect more (bad) weather related problems with trains or tracks during winter.
Let's start by looking at delay as a function of the month of the year.

In [ ]:
vis.plot_train_delays(df=df_otp, kind='month')

Data only runs from March to November 2016. As we can see, there is a large variability observed between months. As a consequence, we will keep this feature, rather than grouping the months into the four seasons.

### Rush hours <a name="Rush_hours"></a>

During rush hours there's an intense increase in the number of passengers and, often, of the number of trains. It is also a period where it is most critical for trains to be on-time.
To identify rush hour periods, we will start by extracting some time features from the dataset: the hour and whether it is a weekend day or not.

In [ ]:
df_otp = sp.extract_time_features(df=df_otp)
df_otp[['train_id', 'origin', 'date', 'hour', 'isWeekend']].sample(5)

Let's now use these features to count the number of distinct trains in the rail system per hour during weekdays, and hence have an idea of the train density per hour.
We define a rush hour as one for which the number of trains per hour and across all hours and weekdays is above the 75th percentile.
Let's visualize the train density per hour on weekdays with that threshold.

In [ ]:
vis.plot_train_density(df=df_otp)

We will now define rush hour as all times on weekdays that go beyond the 75th percentile, which is 135 trains per hour as indicated with the red curve in the plot above. As an example, we will also provide an excerpt of the OTP data.


In [ ]:
df_otp = sp.define_rush_hour(df=df_otp, percentile=75)
df_otp[['train_id', 'origin', 'date', 'hour', 'dayOfTheWeek', 'isRushHour']].sample(5).query("isRushHour==True").head()

### Rank the different stops in a train run <a name="Rank_the_different_stops_in_a_train_run"></a>

The fact that a `train_id` passes only once through a station on a given day allows us to assume that a `train_id` is defined for a specific trip (origin-destination) at a given time in the day. So the combination of a `train_id` and `date` identifies a unique train run (as we defined [before](#Understanding_train_id)). We construct the feature *rank* to identify the sequence number of each stop along the train run, i.e., 1 is origin station, 2 is the second stop, 3 is the third stop, etc.

In [ ]:
df_otp = sp.calculate_rank_stop(df=df_otp)

We can now look at the delay as a function of the rank number.

In [ ]:
vis.plot_delays_per_stop_number(df=df_otp)

We can see that the delay is a function of the train stop number, confirming the idea that trains accumulate delays along the run, without being able of (fully) compensating for them.

### Distance between stations <a name="Distance_between_stations"></a>

We will now use the coordinates information to derive the distance in km between stations, the total distance per train run and the cumulative distance along a train run. Since there are multiple coordinate points for each stop (train position might be registered at different time points, e.g. when entering the station, when docking, when departing, etc.), we will take the average latitude and longitude for each station.

In [ ]:
uni_coords, uni_runs, df_distance, df_coords = sp.calculate_distance_between_stations(df_otp=df_otp, df_train_view=df_train_view)
df_coords[['train_id', 'date', 'rank', 'next_station', 'upcoming_station', 'distance']].sample(5)

We will now visualize an example of a train run. We will plot the locations of all the train stations (in gray) with the stations of the example train run colored by the order they occur.

In [ ]:
vis.plot_train_run(df_train_view=df_train_view, uni_coords=uni_coords,
                   uni_runs=uni_runs, df_distance=df_distance, df_coords=df_coords)

If we plot the distribution of distances between consecutive train stations we can see that most inter-station distances are shorter than 10km.

In [ ]:
vis.plot_distance_distribution(df=df_coords)

### Cumulative distance along a train run <a name="Cumulative_distance_along_a__train__run_"></a>

This feature provides the cumulative distance that a train has traveled along a given train run. It is likely correlated with the rank of a given station on a given train run. Nevertheless, it might convey different information and be useful, depending on the specific questions we will try to address, when applying this feature engineering analysis to a machine learning task.

In the table below we see the information regarding distance and cumulative distance for an example train run.

In [ ]:
df_coords, random_train_run = sp.calculate_cumulative_distance_along_train_run(df=df_coords, uni_runs=uni_runs)
random_train_run

In the scatter plot below we plot the delay on both the distance between two stops and the cumulative distances within the train runs.

In [ ]:
vis.delay_jointplot(df=df_coords, uni_runs=uni_runs)

As we can visually see, in both plots there is no correlation with delay.

### Total distance of a train run <a name="Total_distance_of_a__train__run_"></a>

The total distance traveled by the train is probably closely linked to the total delay or the delay at the last stop of a train run.
Here we will look at train run total distance and delay.

In the first scatter plot below we plot the total delay as a function to the total train run distance. Additionally, we plot the delay at the last stop as a function to the total train run distance.

In [ ]:
vis.total_train_run_distance_joint_plot(df=df_coords)

Also in this case we cannot see a clear relationship between the train run distance and the total delay throughout the run, neither with the delay at the last stop.

In [ ]:
df_otp = sp.add_cum_delay(df=df_otp)
df_otp = sp.add_distance(df_otp=df_otp, df_coords=df_coords)

### Northbound vs Southbound travels <a name="Northbound_vs_Southbound_travels"></a>

This feature identifies stations which are typically problematic, in the sense that there are always delays. We can do this by checking their average delay.

We will first compare the delays of trains operating in one or the other direction.

In [ ]:
vis.plot_northbound_southbound_dely_per_station(df=df_otp)

The asymmetry in the scatterplot (even if not that striking) suggests that delay is dependent on direction (as could be expected, since the distance from origin and stop number will be different according to the direction), so we will label long-duration stations per direction.

### Stations with big delay <a name="Stations_with_big_delay"></a>

Some stations might be more prone to delays. This might be caused e.g. by many tracks converging to a smaller number of platforms of the station or due to long-term works on a station.
We will, thus, rank the average delay for each station and label the top and bottom ones.

In [ ]:
vis.plot_stations_with_long_delays(df=df_otp)

We can label those stations with an average delay above a certain threshold. In this case, we will use the 90th percentile as a threshold. Below you can see the histogram of the average delay per station.

In [ ]:
vis.plot_average_delay(df=df_otp)

sp.print_delays_overview(df=df_otp)
df_otp = sp.label_long_delay_stations(df=df_otp)

The red line on the histogram above indicates the threshold (90th percentile) that was used to label stations with considerably long delays. Most of those stations are in direction S (southbound).

### Delay over the last 7 days <a name="Delay_over_the_last_7_days"></a>

Sometimes, uncontrollable factors can cause delays in trains for a couple of days (e.g. works on the train tracks, long duration strikes, etc.). We can calculate what was the delay over the last 7 days of each date to add that as a feature to predict delays.

In [ ]:
df_otp, last_week_delay = sp.calculate_longest_delays_over_past_week(df=df_otp)
print('10 cases with the longest delay over the past 7 days')
last_week_delay.sort_values('last_week_delay', ascending=False).head(10)

### Track changes <a name="Track_changes"></a>

Track changes can be the cause of delays. To address that, we will compare delays in cases when there was a track change with cases when there was no track changes.

In [ ]:
df_track_changes = sp.get_track_changes(df=df_train_view)
vis.plot_track_changes(df=df_track_changes)

As we can see, delays are higher when there were track changes. A feature we can add to our model is the track change frequency per station per day, defined as:
$$\mathit{Track\space change\space frequency}_{d} = \frac{\sum_{tc\_tr}}{\sum_{tr}}$$
where $d$ is a given date, $tc\_tr$ is a train arrival with a track change and $tr$ is a train.

In [ ]:
track_change_frequency_date = sp.calculate_track_changes_frequency(df=df_track_changes)
track_change_frequency = sp.calculate_aggregated_track_changes(df=track_change_frequency_date)

We can compare how delay changes as a function of track change frequency:

In [ ]:
vis.plot_track_change_frequency_with_delays(df_track_change_frequency=track_change_frequency_date, df_otp=df_otp)

Also in this case, the scatter plot above do not show a clear relationship between delays and track changes.

In the plot below we see the track change frequency of the 10 stations with the highest track change frequency.

In [ ]:
# get unique combinations of station/coordinates and of train_id/dates
uni_coords = df_train_view[['next_station', 'lon', 'lat']].drop_duplicates()
uni_runs = df_otp[['train_id', 'date']].drop_duplicates()

vis.plot_highest_track_change_frequency(track_change_frequency=track_change_frequency)

In the plot below you can see the same figure, but now for the 10 stations with the lowest change frequency. Note that the change frequency for station Cynwyd is zero, which is enforced by the physical limitation of only a single track.

In [ ]:
vis.plot_lowest_track_change_frequency(track_change_frequency)

In [ ]:
# add track change frequency as a feature
df_otp = df_otp.merge(track_change_frequency, on='next_station')

## Conclusion

In this Starter Kit we demonstrated the workflow on how to construct advanced features that are fundamental for answering company business questions such as SEPTA' performance measures. We demonstrated the need of data preparation steps before calculating features as it might simplify the work later. In addition, a wide range of complementary features are constructed, while the impact on train delay is briefly examined using statistical plots.

## Additional information

Copyright © 2022 Sirris

This Starter Kit was developed in the context of the EluciDATA project (http://www.elucidata.be). For more information, please contact info@elucidata.be.

 
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Notebook"), to deal in the Notebook without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Notebook, and to permit persons to whom the Notebook is provided to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies of the Notebook and/or copies of substantial portions of the Notebook.

THE NOTEBOOK IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NON-INFRINGEMENT. IN NO EVENT SHALL SIRRIS, THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, DIRECT OR INDIRECT, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE NOTEBOOK OR THE USE OR OTHER DEALINGS IN THE NOTEBOOK.